In [1]:
import pandas as pd
import numpy as np
import scipy.spatial.distance as ssd
from collections import defaultdict

In [2]:
# 读取训练集，使用到的列为user_id，item_id，rating
df_training_data = pd.read_csv(
    './data/movielen_rating_training.base',
    names=['user_id','item_id','rating'],
    usecols=[0,1,2],
    sep='\t')
df_training_data.head()

,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [3]:
# 不重复的user_id与item_id列表
user_id_s = df_training_data['user_id'].unique()
item_id_s = df_training_data['item_id'].unique()
# 建立id与index的索引
# 例如：user_id_map[【user_id】] = user_index
user_index_map = {}
item_index_map = {}
for user_index in range(len(user_id_s)):
    user_id = user_id_s[user_index]
    user_index_map[user_id] = user_index
for item_index in range(len(item_id_s)):
    item_id = item_id_s[item_index]
    item_index_map[item_id] = item_index

In [4]:
# 用户与物品的打分矩阵
user_item_rating_array = np.zeros(shape=(len(user_id_s),len(item_id_s)))
# 用户打分商品的索引集合
user_rating_map = defaultdict(set)

for row_index in df_training_data.index:
    # 每一行的数据
    row_data = df_training_data.iloc[row_index]
    # 打分用户的索引
    user_index = user_index_map[row_data['user_id']]
    # 打分电影的索引
    item_index = item_index_map[row_data['item_id']]
    # 添加用户打分商品索引集合
    user_rating_map[user_index].add(item_index)
    # 矩阵中行=user_index，列=item_index的元素赋值为打分
    user_item_rating_array[user_index,item_index] = row_data['rating']
user_item_rating_array

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [5]:
# 计算用户的平均打分向量
def calculate_user_rating_mu():
    user_rating_mu_s = []
    for user_index in range(len(user_id_s)):
        # 计算打过分的电影索引
        item_rating_index_v = list(user_rating_map[user_index])
        # 打过分的电影评分向量
        item_rating_v = np.take(
            user_item_rating_array[user_index],
            item_rating_index_v
        )
        # 打分向量的平均值
        mu = item_rating_v.mean()
        # 保留两位小数
        mu = round(mu,2)
        user_rating_mu_s.append(mu)
    return user_rating_mu_s
# ----------------【也可以写成一句话】----------------------------
# def calculate_user_rating_mu():
#     return [
#         round(
#             np.take(
#                 user_item_rating_array[user_index],
#                 list(user_rating_map[user_index])
#             ).mean(),
#             2
#         )
#         for user_index in range(len(user_id_s))
#     ]
# -----------------------------------------------------------------
# 得到用户打分的平均值，列表类型，列表索引对应user_index
user_rating_mu_s = calculate_user_rating_mu()
user_rating_mu_s

[3.68,
 3.8,
 3.0,
 4.36,
 2.96,
 3.58,
 3.89,
 3.6,
 4.17,
 4.21,
 3.53,
 4.28,
 3.14,
 4.22,
 3.03,
 4.35,
 3.16,
 3.94,
 3.6,
 3.31,
 2.66,
 3.3,
 3.64,
 4.39,
 4.05,
 2.91,
 3.3,
 3.64,
 3.94,
 3.8,
 3.91,
 3.55,
 3.64,
 3.8,
 3.0,
 4.0,
 3.46,
 3.9,
 3.85,
 2.73,
 3.94,
 3.62,
 3.68,
 3.64,
 3.48,
 4.18,
 3.56,
 3.73,
 2.73,
 3.54,
 3.75,
 4.34,
 4.0,
 3.33,
 3.8,
 3.65,
 3.63,
 3.88,
 4.02,
 4.13,
 2.83,
 3.32,
 2.98,
 3.6,
 3.98,
 3.52,
 3.42,
 3.17,
 3.74,
 3.4,
 3.82,
 3.77,
 3.65,
 3.76,
 3.18,
 3.4,
 3.46,
 3.36,
 4.19,
 3.93,
 3.57,
 3.13,
 3.4,
 3.74,
 3.55,
 3.64,
 3.78,
 4.09,
 4.15,
 4.25,
 3.9,
 3.23,
 3.0,
 3.66,
 3.48,
 4.25,
 3.97,
 3.88,
 3.65,
 3.03,
 3.0,
 2.56,
 3.4,
 2.79,
 3.31,
 3.71,
 2.77,
 3.41,
 3.5,
 3.09,
 3.5,
 3.92,
 3.81,
 3.63,
 4.04,
 3.06,
 3.72,
 4.64,
 3.88,
 3.57,
 3.67,
 3.97,
 3.93,
 3.5,
 3.44,
 3.7,
 4.18,
 3.58,
 2.87,
 4.08,
 4.07,
 3.71,
 3.28,
 3.58,
 3.45,
 4.55,
 4.08,
 4.3,
 3.93,
 3.6,
 3.45,
 3.8,
 3.73,
 3.67,
 3.32,
 3.68,
 4.3,


In [6]:
# 定义用户相似度函数
def calculate_sim(user_index1,user_index2):
    # 取用户1和用户2公共打分的电影集合，转换成列表
    intersection_index_s = list(
        user_rating_map[user_index1] & user_rating_map[user_index2]
    )
    # 如果没有公共的打分项，相似度为0.0
    if not intersection_index_s:
        return 0.0
    # 根据公共索引，取到用户1的打分向量，并去均值
    v1 = np.take(
        user_item_rating_array[user_index1],
        intersection_index_s
    ) - user_rating_mu_s[user_index1]
    # 根据公共索引，取到用户2的打分向量，并去均值
    v2 = np.take(
        user_item_rating_array[user_index2],
        intersection_index_s
    ) - user_rating_mu_s[user_index2]
    # 计算相似度
    sim = 1 - ssd.cosine(v1,v2)
    # 如果相似度不是数字（如果v1或v2是0向量），返回相似度0
    if np.isnan(sim):
        return 0.0
    # 否则相似度保留两位小数，返回结果
    else:
        return round(sim,2)

In [45]:
user_similarity_array = np.zeros(shape=(len(user_id_s),len(user_id_s)))
for user_index1 in range(len(user_id_s)):
    print('计算到用户：%s与其余用户的相似度' % user_index1)
    for user_index2 in range(user_index1 + 1,len(user_id_s)):
        # 计算用户1和用户2的相似度
        sim = calculate_sim(user_index1,user_index2)
        # 用户1与用户2的相似度=sim
        user_similarity_array[user_index1,user_index2] = sim
        # 用户2与用户1的相似度=sim
        user_similarity_array[user_index2,user_index1] = sim

计算到用户：0与其余用户的相似度
计算到用户：1与其余用户的相似度
计算到用户：2与其余用户的相似度
计算到用户：3与其余用户的相似度
计算到用户：4与其余用户的相似度
计算到用户：5与其余用户的相似度
计算到用户：6与其余用户的相似度
计算到用户：7与其余用户的相似度
计算到用户：8与其余用户的相似度
计算到用户：9与其余用户的相似度
计算到用户：10与其余用户的相似度
计算到用户：11与其余用户的相似度
计算到用户：12与其余用户的相似度
计算到用户：13与其余用户的相似度
计算到用户：14与其余用户的相似度
计算到用户：15与其余用户的相似度
计算到用户：16与其余用户的相似度
计算到用户：17与其余用户的相似度
计算到用户：18与其余用户的相似度
计算到用户：19与其余用户的相似度
计算到用户：20与其余用户的相似度
计算到用户：21与其余用户的相似度
计算到用户：22与其余用户的相似度
计算到用户：23与其余用户的相似度
计算到用户：24与其余用户的相似度
计算到用户：25与其余用户的相似度
计算到用户：26与其余用户的相似度
计算到用户：27与其余用户的相似度
计算到用户：28与其余用户的相似度
计算到用户：29与其余用户的相似度
计算到用户：30与其余用户的相似度
计算到用户：31与其余用户的相似度
计算到用户：32与其余用户的相似度
计算到用户：33与其余用户的相似度
计算到用户：34与其余用户的相似度
计算到用户：35与其余用户的相似度
计算到用户：36与其余用户的相似度
计算到用户：37与其余用户的相似度
计算到用户：38与其余用户的相似度
计算到用户：39与其余用户的相似度
计算到用户：40与其余用户的相似度
计算到用户：41与其余用户的相似度
计算到用户：42与其余用户的相似度
计算到用户：43与其余用户的相似度
计算到用户：44与其余用户的相似度
计算到用户：45与其余用户的相似度
计算到用户：46与其余用户的相似度
计算到用户：47与其余用户的相似度
计算到用户：48与其余用户的相似度
计算到用户：49与其余用户的相似度
计算到用户：50与其余用户的相似度
计算到用户：51与其余用户的相似度
计算到用户：52与其余用户的相似度
计算到用户：53与其余用户的相似度
计算到用户：54与其余用户的相似度
计算到用户：55与其余用户的相似度
计算

计算到用户：439与其余用户的相似度
计算到用户：440与其余用户的相似度
计算到用户：441与其余用户的相似度
计算到用户：442与其余用户的相似度
计算到用户：443与其余用户的相似度
计算到用户：444与其余用户的相似度
计算到用户：445与其余用户的相似度
计算到用户：446与其余用户的相似度
计算到用户：447与其余用户的相似度
计算到用户：448与其余用户的相似度
计算到用户：449与其余用户的相似度
计算到用户：450与其余用户的相似度
计算到用户：451与其余用户的相似度
计算到用户：452与其余用户的相似度
计算到用户：453与其余用户的相似度
计算到用户：454与其余用户的相似度
计算到用户：455与其余用户的相似度
计算到用户：456与其余用户的相似度
计算到用户：457与其余用户的相似度
计算到用户：458与其余用户的相似度
计算到用户：459与其余用户的相似度
计算到用户：460与其余用户的相似度
计算到用户：461与其余用户的相似度
计算到用户：462与其余用户的相似度
计算到用户：463与其余用户的相似度
计算到用户：464与其余用户的相似度
计算到用户：465与其余用户的相似度
计算到用户：466与其余用户的相似度
计算到用户：467与其余用户的相似度
计算到用户：468与其余用户的相似度
计算到用户：469与其余用户的相似度
计算到用户：470与其余用户的相似度
计算到用户：471与其余用户的相似度
计算到用户：472与其余用户的相似度
计算到用户：473与其余用户的相似度
计算到用户：474与其余用户的相似度
计算到用户：475与其余用户的相似度
计算到用户：476与其余用户的相似度
计算到用户：477与其余用户的相似度
计算到用户：478与其余用户的相似度
计算到用户：479与其余用户的相似度
计算到用户：480与其余用户的相似度
计算到用户：481与其余用户的相似度
计算到用户：482与其余用户的相似度
计算到用户：483与其余用户的相似度
计算到用户：484与其余用户的相似度
计算到用户：485与其余用户的相似度
计算到用户：486与其余用户的相似度
计算到用户：487与其余用户的相似度
计算到用户：488与其余用户的相似度
计算到用户：489与其余用户的相似度
计算到用户：490与其余用户的相似度
计算到用户：491与其余

计算到用户：875与其余用户的相似度
计算到用户：876与其余用户的相似度
计算到用户：877与其余用户的相似度
计算到用户：878与其余用户的相似度
计算到用户：879与其余用户的相似度
计算到用户：880与其余用户的相似度
计算到用户：881与其余用户的相似度
计算到用户：882与其余用户的相似度
计算到用户：883与其余用户的相似度
计算到用户：884与其余用户的相似度
计算到用户：885与其余用户的相似度
计算到用户：886与其余用户的相似度
计算到用户：887与其余用户的相似度
计算到用户：888与其余用户的相似度
计算到用户：889与其余用户的相似度
计算到用户：890与其余用户的相似度
计算到用户：891与其余用户的相似度
计算到用户：892与其余用户的相似度
计算到用户：893与其余用户的相似度
计算到用户：894与其余用户的相似度
计算到用户：895与其余用户的相似度
计算到用户：896与其余用户的相似度
计算到用户：897与其余用户的相似度
计算到用户：898与其余用户的相似度
计算到用户：899与其余用户的相似度
计算到用户：900与其余用户的相似度
计算到用户：901与其余用户的相似度
计算到用户：902与其余用户的相似度
计算到用户：903与其余用户的相似度
计算到用户：904与其余用户的相似度
计算到用户：905与其余用户的相似度
计算到用户：906与其余用户的相似度
计算到用户：907与其余用户的相似度
计算到用户：908与其余用户的相似度
计算到用户：909与其余用户的相似度
计算到用户：910与其余用户的相似度
计算到用户：911与其余用户的相似度
计算到用户：912与其余用户的相似度
计算到用户：913与其余用户的相似度
计算到用户：914与其余用户的相似度
计算到用户：915与其余用户的相似度
计算到用户：916与其余用户的相似度
计算到用户：917与其余用户的相似度
计算到用户：918与其余用户的相似度
计算到用户：919与其余用户的相似度
计算到用户：920与其余用户的相似度
计算到用户：921与其余用户的相似度
计算到用户：922与其余用户的相似度
计算到用户：923与其余用户的相似度
计算到用户：924与其余用户的相似度
计算到用户：925与其余用户的相似度
计算到用户：926与其余用户的相似度
计算到用户：927与其余

In [8]:
# 打印用户相似度矩阵
user_similarity_array

array([[ 0.  ,  0.52,  0.53, ...,  0.81, -0.18,  0.3 ],
       [ 0.52,  0.  ,  0.14, ...,  0.04,  0.25,  0.71],
       [ 0.53,  0.14,  0.  , ...,  0.74,  0.51,  1.  ],
       ...,
       [ 0.81,  0.04,  0.74, ...,  0.  , -0.46,  0.01],
       [-0.18,  0.25,  0.51, ..., -0.46,  0.  ,  0.42],
       [ 0.3 ,  0.71,  1.  , ...,  0.01,  0.42,  0.  ]])

In [9]:
# 用户对商品的预测矩阵，已打分的商品，预测分数为0分
user_item_predict_rating_array = np.zeros_like(user_item_rating_array)

# 对所有的用户index进行遍历
for user_index in range(len(user_id_s)):
    print('预测到用户%s' % user_index)
    # 对所有商品进行遍历
    for item_index in range(len(item_id_s)):
        # 如果这个商品没有被打过分
        if item_index not in user_rating_map[user_index]:
            # 找到对这个商品打过分的所有用户的索引
            user_rating_index_v = np.where(
                user_item_rating_array[:,item_index] > 0
            )[0]
            # 如果没有用户对这个商品打过分，continue
            if not list(user_rating_index_v):
                continue
            # 根据用户打分的索引，从用户相似度矩阵中取出相似度向量
            user_sim_v = np.take(
                user_similarity_array[user_index],
                user_rating_index_v,
            )
            # 计算相似度绝对值加和
            user_sim_abs_sum = user_sim_v.__abs__().sum()
            # 如果相似度绝对值加和为0（也就是存在对这个商品打过分的用户群体，但这些用户群体与目标用户的相关度都为0），continue
            if user_sim_abs_sum == 0:
                continue
            # 得到用户打分的向量，并去掉平均值
            user_rating_v = np.take(
                user_item_rating_array[:,item_index],
                user_rating_index_v
            ) - np.take(
                user_rating_mu_s,
                user_rating_index_v
            )
            # 根据公式，得到预测的结果，这里保留两位小数
            predict_rating = round(
                (np.dot(user_rating_v,user_sim_v) + user_rating_mu_s[user_index])/user_sim_abs_sum
                ,2
            ) + user_rating_mu_s[user_index]
            # 把预测的结果添加到预测矩阵中
            user_item_predict_rating_array[user_index,item_index] = predict_rating
# 打印预测矩阵
user_item_predict_rating_array    

预测到用户0
预测到用户1
预测到用户2
预测到用户3
预测到用户4
预测到用户5
预测到用户6
预测到用户7
预测到用户8
预测到用户9
预测到用户10
预测到用户11
预测到用户12
预测到用户13
预测到用户14
预测到用户15
预测到用户16
预测到用户17
预测到用户18
预测到用户19
预测到用户20
预测到用户21
预测到用户22
预测到用户23
预测到用户24
预测到用户25
预测到用户26
预测到用户27
预测到用户28
预测到用户29
预测到用户30
预测到用户31
预测到用户32
预测到用户33
预测到用户34
预测到用户35
预测到用户36
预测到用户37
预测到用户38
预测到用户39
预测到用户40
预测到用户41
预测到用户42
预测到用户43
预测到用户44
预测到用户45
预测到用户46
预测到用户47
预测到用户48
预测到用户49
预测到用户50
预测到用户51
预测到用户52
预测到用户53
预测到用户54
预测到用户55
预测到用户56
预测到用户57
预测到用户58
预测到用户59
预测到用户60
预测到用户61
预测到用户62
预测到用户63
预测到用户64
预测到用户65
预测到用户66
预测到用户67
预测到用户68
预测到用户69
预测到用户70
预测到用户71
预测到用户72
预测到用户73
预测到用户74
预测到用户75
预测到用户76
预测到用户77
预测到用户78
预测到用户79
预测到用户80
预测到用户81
预测到用户82
预测到用户83
预测到用户84
预测到用户85
预测到用户86
预测到用户87
预测到用户88
预测到用户89
预测到用户90
预测到用户91
预测到用户92
预测到用户93
预测到用户94
预测到用户95
预测到用户96
预测到用户97
预测到用户98
预测到用户99
预测到用户100
预测到用户101
预测到用户102
预测到用户103
预测到用户104
预测到用户105
预测到用户106
预测到用户107
预测到用户108
预测到用户109
预测到用户110
预测到用户111
预测到用户112
预测到用户113
预测到用户114
预测到用户115
预测到用户116
预测到用户117
预测到用户118
预测到用户119
预测到用户120
预测到用户121
预测到用户122
预测到

预测到用户923
预测到用户924
预测到用户925
预测到用户926
预测到用户927
预测到用户928
预测到用户929
预测到用户930
预测到用户931
预测到用户932
预测到用户933
预测到用户934
预测到用户935
预测到用户936
预测到用户937
预测到用户938
预测到用户939
预测到用户940
预测到用户941
预测到用户942


array([[  0.  ,   0.  ,   0.  , ...,  13.4 ,  12.68,  10.98],
       [  0.  ,   3.62,   3.63, ..., 193.3 ,  24.93,  11.52],
       [  3.09,   3.02,   2.75, ..., 103.5 ,  19.69,   0.  ],
       ...,
       [  0.  ,   3.91,   3.91, ...,  18.05,  25.39,  21.29],
       [  4.54,   4.28,   4.19, ...,  25.12,  17.23,  20.32],
       [  3.7 ,   0.  ,   3.06, ...,   8.89,  15.19,  13.7 ]])

In [10]:
# 读取测试集
df_test_data = pd.read_csv('./data/movielen_rating_test.base',sep='\t',names=['user_id','item_id','rating'],usecols=[0,1,2])
df_test_data.head()

,user_id,item_id,rating
0,1,6,5
1,1,10,3
2,1,12,5
3,1,14,5
4,1,17,3


In [11]:
# 测试集中不重复的用户id
user_test_unique_s = df_test_data['user_id'].unique()
# 创建一个列表，保存测试集中的user_id对应的user_index
user_index_test_s = []
# 对测试集中的用户id进行遍历
for user_id in user_test_unique_s:
    # 如果测试集中的用户id在训练集的用户索引map中，添加这个user_index
    if user_id in user_index_map.keys():
        user_index_test_s.append(user_index_map[user_id])
# 打印测试集中的user_index列表
user_index_test_s

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [46]:
# 创建一个用户对商品打分的dataframe
df_user_item_rating_test = pd.DataFrame(np.zeros(shape=(len(user_index_test_s),len(item_id_s))))
# dataframe的index设置为在训练集中存在的测试集的user_index
df_user_item_rating_test.index = user_index_test_s
# 对dataframe的index进行遍历
for row_index in df_test_data.index:
    print('运行到%s行'% row_index)
    row_data = df_test_data.loc[row_index]
    # 如果这个用户id在user_rating_map中，商品id也在item_index_map中，添加这次打分
    if row_data['user_id'] in user_rating_map.keys() and row_data['item_id'] in item_index_map.keys():
        df_user_item_rating_test[item_index_map[row_data['item_id']]][user_index_map[row_data['user_id']]] = row_data['rating']
# 打印dataframe
df_user_item_rating_test

运行到0行
运行到1行
运行到2行
运行到3行
运行到4行
运行到5行
运行到6行
运行到7行
运行到8行
运行到9行
运行到10行
运行到11行
运行到12行
运行到13行
运行到14行
运行到15行
运行到16行
运行到17行
运行到18行
运行到19行
运行到20行
运行到21行
运行到22行
运行到23行
运行到24行
运行到25行
运行到26行
运行到27行
运行到28行
运行到29行
运行到30行
运行到31行
运行到32行
运行到33行
运行到34行
运行到35行
运行到36行
运行到37行
运行到38行
运行到39行
运行到40行
运行到41行
运行到42行
运行到43行
运行到44行
运行到45行
运行到46行
运行到47行
运行到48行
运行到49行
运行到50行
运行到51行
运行到52行
运行到53行
运行到54行
运行到55行
运行到56行
运行到57行
运行到58行
运行到59行
运行到60行
运行到61行
运行到62行
运行到63行
运行到64行
运行到65行
运行到66行
运行到67行
运行到68行
运行到69行
运行到70行
运行到71行
运行到72行
运行到73行
运行到74行
运行到75行
运行到76行
运行到77行
运行到78行
运行到79行
运行到80行
运行到81行
运行到82行
运行到83行
运行到84行
运行到85行
运行到86行
运行到87行
运行到88行
运行到89行
运行到90行
运行到91行
运行到92行
运行到93行
运行到94行
运行到95行
运行到96行
运行到97行
运行到98行
运行到99行
运行到100行
运行到101行
运行到102行
运行到103行
运行到104行
运行到105行
运行到106行
运行到107行
运行到108行
运行到109行
运行到110行
运行到111行
运行到112行
运行到113行
运行到114行
运行到115行
运行到116行
运行到117行
运行到118行
运行到119行
运行到120行
运行到121行
运行到122行
运行到123行
运行到124行
运行到125行
运行到126行
运行到127行
运行到128行
运行到129行
运行到130行
运行到131行
运行到132行
运行到133行
运行到134行
运行到135行
运行到136行
运行到137行
运行到138

运行到1326行
运行到1327行
运行到1328行
运行到1329行
运行到1330行
运行到1331行
运行到1332行
运行到1333行
运行到1334行
运行到1335行
运行到1336行
运行到1337行
运行到1338行
运行到1339行
运行到1340行
运行到1341行
运行到1342行
运行到1343行
运行到1344行
运行到1345行
运行到1346行
运行到1347行
运行到1348行
运行到1349行
运行到1350行
运行到1351行
运行到1352行
运行到1353行
运行到1354行
运行到1355行
运行到1356行
运行到1357行
运行到1358行
运行到1359行
运行到1360行
运行到1361行
运行到1362行
运行到1363行
运行到1364行
运行到1365行
运行到1366行
运行到1367行
运行到1368行
运行到1369行
运行到1370行
运行到1371行
运行到1372行
运行到1373行
运行到1374行
运行到1375行
运行到1376行
运行到1377行
运行到1378行
运行到1379行
运行到1380行
运行到1381行
运行到1382行
运行到1383行
运行到1384行
运行到1385行
运行到1386行
运行到1387行
运行到1388行
运行到1389行
运行到1390行
运行到1391行
运行到1392行
运行到1393行
运行到1394行
运行到1395行
运行到1396行
运行到1397行
运行到1398行
运行到1399行
运行到1400行
运行到1401行
运行到1402行
运行到1403行
运行到1404行
运行到1405行
运行到1406行
运行到1407行
运行到1408行
运行到1409行
运行到1410行
运行到1411行
运行到1412行
运行到1413行
运行到1414行
运行到1415行
运行到1416行
运行到1417行
运行到1418行
运行到1419行
运行到1420行
运行到1421行
运行到1422行
运行到1423行
运行到1424行
运行到1425行
运行到1426行
运行到1427行
运行到1428行
运行到1429行
运行到1430行
运行到1431行
运行到1432行
运行到1433行
运行到1434行
运行到1435行
运行到1436行
运

运行到2445行
运行到2446行
运行到2447行
运行到2448行
运行到2449行
运行到2450行
运行到2451行
运行到2452行
运行到2453行
运行到2454行
运行到2455行
运行到2456行
运行到2457行
运行到2458行
运行到2459行
运行到2460行
运行到2461行
运行到2462行
运行到2463行
运行到2464行
运行到2465行
运行到2466行
运行到2467行
运行到2468行
运行到2469行
运行到2470行
运行到2471行
运行到2472行
运行到2473行
运行到2474行
运行到2475行
运行到2476行
运行到2477行
运行到2478行
运行到2479行
运行到2480行
运行到2481行
运行到2482行
运行到2483行
运行到2484行
运行到2485行
运行到2486行
运行到2487行
运行到2488行
运行到2489行
运行到2490行
运行到2491行
运行到2492行
运行到2493行
运行到2494行
运行到2495行
运行到2496行
运行到2497行
运行到2498行
运行到2499行
运行到2500行
运行到2501行
运行到2502行
运行到2503行
运行到2504行
运行到2505行
运行到2506行
运行到2507行
运行到2508行
运行到2509行
运行到2510行
运行到2511行
运行到2512行
运行到2513行
运行到2514行
运行到2515行
运行到2516行
运行到2517行
运行到2518行
运行到2519行
运行到2520行
运行到2521行
运行到2522行
运行到2523行
运行到2524行
运行到2525行
运行到2526行
运行到2527行
运行到2528行
运行到2529行
运行到2530行
运行到2531行
运行到2532行
运行到2533行
运行到2534行
运行到2535行
运行到2536行
运行到2537行
运行到2538行
运行到2539行
运行到2540行
运行到2541行
运行到2542行
运行到2543行
运行到2544行
运行到2545行
运行到2546行
运行到2547行
运行到2548行
运行到2549行
运行到2550行
运行到2551行
运行到2552行
运行到2553行
运行到2554行
运行到2555行
运

运行到3577行
运行到3578行
运行到3579行
运行到3580行
运行到3581行
运行到3582行
运行到3583行
运行到3584行
运行到3585行
运行到3586行
运行到3587行
运行到3588行
运行到3589行
运行到3590行
运行到3591行
运行到3592行
运行到3593行
运行到3594行
运行到3595行
运行到3596行
运行到3597行
运行到3598行
运行到3599行
运行到3600行
运行到3601行
运行到3602行
运行到3603行
运行到3604行
运行到3605行
运行到3606行
运行到3607行
运行到3608行
运行到3609行
运行到3610行
运行到3611行
运行到3612行
运行到3613行
运行到3614行
运行到3615行
运行到3616行
运行到3617行
运行到3618行
运行到3619行
运行到3620行
运行到3621行
运行到3622行
运行到3623行
运行到3624行
运行到3625行
运行到3626行
运行到3627行
运行到3628行
运行到3629行
运行到3630行
运行到3631行
运行到3632行
运行到3633行
运行到3634行
运行到3635行
运行到3636行
运行到3637行
运行到3638行
运行到3639行
运行到3640行
运行到3641行
运行到3642行
运行到3643行
运行到3644行
运行到3645行
运行到3646行
运行到3647行
运行到3648行
运行到3649行
运行到3650行
运行到3651行
运行到3652行
运行到3653行
运行到3654行
运行到3655行
运行到3656行
运行到3657行
运行到3658行
运行到3659行
运行到3660行
运行到3661行
运行到3662行
运行到3663行
运行到3664行
运行到3665行
运行到3666行
运行到3667行
运行到3668行
运行到3669行
运行到3670行
运行到3671行
运行到3672行
运行到3673行
运行到3674行
运行到3675行
运行到3676行
运行到3677行
运行到3678行
运行到3679行
运行到3680行
运行到3681行
运行到3682行
运行到3683行
运行到3684行
运行到3685行
运行到3686行
运行到3687行
运

运行到4517行
运行到4518行
运行到4519行
运行到4520行
运行到4521行
运行到4522行
运行到4523行
运行到4524行
运行到4525行
运行到4526行
运行到4527行
运行到4528行
运行到4529行
运行到4530行
运行到4531行
运行到4532行
运行到4533行
运行到4534行
运行到4535行
运行到4536行
运行到4537行
运行到4538行
运行到4539行
运行到4540行
运行到4541行
运行到4542行
运行到4543行
运行到4544行
运行到4545行
运行到4546行
运行到4547行
运行到4548行
运行到4549行
运行到4550行
运行到4551行
运行到4552行
运行到4553行
运行到4554行
运行到4555行
运行到4556行
运行到4557行
运行到4558行
运行到4559行
运行到4560行
运行到4561行
运行到4562行
运行到4563行
运行到4564行
运行到4565行
运行到4566行
运行到4567行
运行到4568行
运行到4569行
运行到4570行
运行到4571行
运行到4572行
运行到4573行
运行到4574行
运行到4575行
运行到4576行
运行到4577行
运行到4578行
运行到4579行
运行到4580行
运行到4581行
运行到4582行
运行到4583行
运行到4584行
运行到4585行
运行到4586行
运行到4587行
运行到4588行
运行到4589行
运行到4590行
运行到4591行
运行到4592行
运行到4593行
运行到4594行
运行到4595行
运行到4596行
运行到4597行
运行到4598行
运行到4599行
运行到4600行
运行到4601行
运行到4602行
运行到4603行
运行到4604行
运行到4605行
运行到4606行
运行到4607行
运行到4608行
运行到4609行
运行到4610行
运行到4611行
运行到4612行
运行到4613行
运行到4614行
运行到4615行
运行到4616行
运行到4617行
运行到4618行
运行到4619行
运行到4620行
运行到4621行
运行到4622行
运行到4623行
运行到4624行
运行到4625行
运行到4626行
运行到4627行
运

...

运行到15974行
运行到15975行
运行到15976行
运行到15977行
运行到15978行
运行到15979行
运行到15980行
运行到15981行
运行到15982行
运行到15983行
运行到15984行
运行到15985行
运行到15986行
运行到15987行
运行到15988行
运行到15989行
运行到15990行
运行到15991行
运行到15992行
运行到15993行
运行到15994行
运行到15995行
运行到15996行
运行到15997行
运行到15998行
运行到15999行
运行到16000行
运行到16001行
运行到16002行
运行到16003行
运行到16004行
运行到16005行
运行到16006行
运行到16007行
运行到16008行
运行到16009行
运行到16010行
运行到16011行
运行到16012行
运行到16013行
运行到16014行
运行到16015行
运行到16016行
运行到16017行
运行到16018行
运行到16019行
运行到16020行
运行到16021行
运行到16022行
运行到16023行
运行到16024行
运行到16025行
运行到16026行
运行到16027行
运行到16028行
运行到16029行
运行到16030行
运行到16031行
运行到16032行
运行到16033行
运行到16034行
运行到16035行
运行到16036行
运行到16037行
运行到16038行
运行到16039行
运行到16040行
运行到16041行
运行到16042行
运行到16043行
运行到16044行
运行到16045行
运行到16046行
运行到16047行
运行到16048行
运行到16049行
运行到16050行
运行到16051行
运行到16052行
运行到16053行
运行到16054行
运行到16055行
运行到16056行
运行到16057行
运行到16058行
运行到16059行
运行到16060行
运行到16061行
运行到16062行
运行到16063行
运行到16064行
运行到16065行
运行到16066行
运行到16067行
运行到16068行
运行到16069行
运行到16070行
运行到16071行
运行到16072行
运行到16073行


运行到17081行
运行到17082行
运行到17083行
运行到17084行
运行到17085行
运行到17086行
运行到17087行
运行到17088行
运行到17089行
运行到17090行
运行到17091行
运行到17092行
运行到17093行
运行到17094行
运行到17095行
运行到17096行
运行到17097行
运行到17098行
运行到17099行
运行到17100行
运行到17101行
运行到17102行
运行到17103行
运行到17104行
运行到17105行
运行到17106行
运行到17107行
运行到17108行
运行到17109行
运行到17110行
运行到17111行
运行到17112行
运行到17113行
运行到17114行
运行到17115行
运行到17116行
运行到17117行
运行到17118行
运行到17119行
运行到17120行
运行到17121行
运行到17122行
运行到17123行
运行到17124行
运行到17125行
运行到17126行
运行到17127行
运行到17128行
运行到17129行
运行到17130行
运行到17131行
运行到17132行
运行到17133行
运行到17134行
运行到17135行
运行到17136行
运行到17137行
运行到17138行
运行到17139行
运行到17140行
运行到17141行
运行到17142行
运行到17143行
运行到17144行
运行到17145行
运行到17146行
运行到17147行
运行到17148行
运行到17149行
运行到17150行
运行到17151行
运行到17152行
运行到17153行
运行到17154行
运行到17155行
运行到17156行
运行到17157行
运行到17158行
运行到17159行
运行到17160行
运行到17161行
运行到17162行
运行到17163行
运行到17164行
运行到17165行
运行到17166行
运行到17167行
运行到17168行
运行到17169行
运行到17170行
运行到17171行
运行到17172行
运行到17173行
运行到17174行
运行到17175行
运行到17176行
运行到17177行
运行到17178行
运行到17179行
运行到17180行


运行到18204行
运行到18205行
运行到18206行
运行到18207行
运行到18208行
运行到18209行
运行到18210行
运行到18211行
运行到18212行
运行到18213行
运行到18214行
运行到18215行
运行到18216行
运行到18217行
运行到18218行
运行到18219行
运行到18220行
运行到18221行
运行到18222行
运行到18223行
运行到18224行
运行到18225行
运行到18226行
运行到18227行
运行到18228行
运行到18229行
运行到18230行
运行到18231行
运行到18232行
运行到18233行
运行到18234行
运行到18235行
运行到18236行
运行到18237行
运行到18238行
运行到18239行
运行到18240行
运行到18241行
运行到18242行
运行到18243行
运行到18244行
运行到18245行
运行到18246行
运行到18247行
运行到18248行
运行到18249行
运行到18250行
运行到18251行
运行到18252行
运行到18253行
运行到18254行
运行到18255行
运行到18256行
运行到18257行
运行到18258行
运行到18259行
运行到18260行
运行到18261行
运行到18262行
运行到18263行
运行到18264行
运行到18265行
运行到18266行
运行到18267行
运行到18268行
运行到18269行
运行到18270行
运行到18271行
运行到18272行
运行到18273行
运行到18274行
运行到18275行
运行到18276行
运行到18277行
运行到18278行
运行到18279行
运行到18280行
运行到18281行
运行到18282行
运行到18283行
运行到18284行
运行到18285行
运行到18286行
运行到18287行
运行到18288行
运行到18289行
运行到18290行
运行到18291行
运行到18292行
运行到18293行
运行到18294行
运行到18295行
运行到18296行
运行到18297行
运行到18298行
运行到18299行
运行到18300行
运行到18301行
运行到18302行
运行到18303行


运行到19034行
运行到19035行
运行到19036行
运行到19037行
运行到19038行
运行到19039行
运行到19040行
运行到19041行
运行到19042行
运行到19043行
运行到19044行
运行到19045行
运行到19046行
运行到19047行
运行到19048行
运行到19049行
运行到19050行
运行到19051行
运行到19052行
运行到19053行
运行到19054行
运行到19055行
运行到19056行
运行到19057行
运行到19058行
运行到19059行
运行到19060行
运行到19061行
运行到19062行
运行到19063行
运行到19064行
运行到19065行
运行到19066行
运行到19067行
运行到19068行
运行到19069行
运行到19070行
运行到19071行
运行到19072行
运行到19073行
运行到19074行
运行到19075行
运行到19076行
运行到19077行
运行到19078行
运行到19079行
运行到19080行
运行到19081行
运行到19082行
运行到19083行
运行到19084行
运行到19085行
运行到19086行
运行到19087行
运行到19088行
运行到19089行
运行到19090行
运行到19091行
运行到19092行
运行到19093行
运行到19094行
运行到19095行
运行到19096行
运行到19097行
运行到19098行
运行到19099行
运行到19100行
运行到19101行
运行到19102行
运行到19103行
运行到19104行
运行到19105行
运行到19106行
运行到19107行
运行到19108行
运行到19109行
运行到19110行
运行到19111行
运行到19112行
运行到19113行
运行到19114行
运行到19115行
运行到19116行
运行到19117行
运行到19118行
运行到19119行
运行到19120行
运行到19121行
运行到19122行
运行到19123行
运行到19124行
运行到19125行
运行到19126行
运行到19127行
运行到19128行
运行到19129行
运行到19130行
运行到19131行
运行到19132行
运行到19133行


,0,1,2,3,4,5,6,7,8,9,...,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# 计算均方误差
def calculate_RMSE():
    # acc_locc为分子，二次损失
    acc_loss = 0
    # acc_num为分母，一共计算了多少项
    acc_num = 0
    # 对测试集的index进行遍历
    for user_index in df_user_item_rating_test.index:
        # 测试集中user_index对应的行向量
        test_row_data = np.array(df_user_item_rating_test.loc[user_index])
        # 预测矩阵中的行向量
        predict_row_data = user_item_predict_rating_array[user_index]
        # 测试集中对应user_index，打过分的商品索引
        test_index_v =  np.where(test_row_data > 0)
        # 预测矩阵中对应user_index，打过分的商品索引
        predict_index_v = np.where(predict_row_data > 0)
        # 取test_index_v和predict_index_v的交集，即预测过打分，而且也在测试集中出现实际打分
        intersection_index_s = list(
            set(test_index_v[0]) & set(predict_index_v[0])
        )
        # 如果交集为空，continue
        if not intersection_index_s:
            continue
        # 根据上述的交集索引，取得测试集中的打分向量和预测矩阵中的打分向量
        test_rating_v = np.take(test_row_data,intersection_index_s)[0]
        predict_rating_v = np.clip(
            np.take(predict_row_data,intersection_index_s)[0],0,5
        )
        # 计算二次损失
        acc_loss += np.square(test_rating_v - predict_rating_v)
        # 分母叠加个数
        acc_num += len(intersection_index_s)
    # 得出均方误差
    return np.sqrt(acc_loss/acc_num)
# 计算均方误差并打印
RMSE = calculate_RMSE()
RMSE

0.13892020392313634

In [39]:
# 推荐商品，predict_quantity是推荐的商品的个数
def predict(predict_quantity):
    # 建立一个商品推荐字典,保存对user_index推荐的商品索引
    # 例如：predict_item_index_map[user_index] = [4,3,2,5] 
    predict_item_index_map = {}
    # 对训练集中所有的user_index进行遍历
    for user_index in range(len(user_id_s)):
        # 预测矩阵中对应user_index的向量，进行倒序排列
        predict_item_index_v = list(np.argsort(-user_item_predict_rating_array[user_index]))
        # 取min(min(predict_quantity,len(predict_item_index_v)))个最前面的商品索引，即打分最高的商品索引
        predict_item_index_v = predict_item_index_v[0:min(predict_quantity,len(predict_item_index_v))]
        # 添加到商品推荐字典中
        predict_item_index_map[user_index] = predict_item_index_v
    # 返回商品推荐字典
    return predict_item_index_map
# 推荐50个商品
predict_item_index_map = predict(50)
# 打印商品推荐字典
predict_item_index_map

{0: [1488,
  1478,
  1479,
  1480,
  1481,
  1482,
  1484,
  1476,
  1485,
  1487,
  1489,
  1490,
  1491,
  1492,
  1494,
  1486,
  1475,
  1477,
  1443,
  1452,
  1466,
  1447,
  1464,
  1450,
  1473,
  1451,
  1640,
  1210,
  1215,
  1216,
  1227,
  1219,
  1217,
  1220,
  1221,
  1229,
  1207,
  1518,
  1226,
  1184,
  1445,
  1324,
  1225,
  1561,
  1570,
  1514,
  1610,
  1555,
  770,
  1645],
 1: [1644,
  1646,
  1045,
  1645,
  1492,
  1482,
  1481,
  1480,
  1479,
  1464,
  1478,
  1452,
  1477,
  1476,
  1473,
  1466,
  1484,
  1486,
  1475,
  1487,
  1488,
  1489,
  1490,
  1451,
  1450,
  1491,
  1447,
  1443,
  1494,
  1485,
  952,
  1265,
  1647,
  1573,
  1568,
  1552,
  1567,
  1324,
  1528,
  1229,
  1210,
  1220,
  1221,
  1207,
  1219,
  1216,
  1227,
  1217,
  1215,
  1465],
 2: [1522,
  787,
  1644,
  1646,
  1647,
  1645,
  1385,
  1505,
  1561,
  1543,
  1639,
  1449,
  1272,
  1514,
  1612,
  1320,
  1322,
  1560,
  1431,
  1321,
  1315,
  1318,
  1396,
  1319,


In [41]:
# 计算准确率与召回率
def calculate_precision_and_recall():
    # 推荐的商品也在测试集中出现的总数
    union_num = 0
    # 推荐的商品的总数
    predict_num = 0
    # 测试集中出现的商品总数
    test_num = 0
    # 对测试集的user_index进行遍历
    for user_index in df_user_item_rating_test.index:
        # 对测试集中的user_index打过分的商品，进行倒排序，得到索引
        #（这里其实没有使用到倒序排列功能，比如可以取测试集中倒序的前100个所以test_item_v）
        test_item_v = list(
            np.argsort(
                -np.array(
                    df_user_item_rating_test[user_index]
                )
            )
        )
        # 推荐的商品也在测试集中出现的总数做叠加
        union_num += len(
            set(predict_item_index_map[user_index]) & set(test_item_v)
        )
        # 推荐的商品的总数做叠加
        predict_num += len(predict_item_index_map[user_index])
        # 测试集中出现的商品总数做叠加
        test_num += len(test_item_v)
    # 返回准确率与召回率
    return union_num / predict_num,union_num/test_num
# 计算准确率与召回率
precision,recall = calculate_precision_and_recall()
print('precision=',precision)
print('recall=',recall)

precision= 0.0044444444444444444
recall= 0.0004841442749939482


In [43]:
# 定义计算覆盖率
def calculate_coverage():
    # 推荐的物品索引集合
    predict_item_index_set = set()
    # 把所有用户推荐过的商品id都添加到predict_item_index_set里，然后根据predict_item_index_set的数量，计算覆盖度
    for user_index in predict_item_index_map.keys():
        for item_index in predict_item_index_map[user_index]:
            predict_item_index_set.add(item_index)
    return len(predict_item_index_set) / len(item_id_s)
# 计算并打印覆盖度
coverage = calculate_coverage()
print('coverage=',coverage)

coverage= 0.23757575757575758
